In [1]:
import requests
import json
import pandas as pd

In [7]:
def get_response_from_server(prompt, model):
    url = "http://localhost:11434/api/chat"
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }

    try:
        response = requests.post(url, json=payload, stream=True)
        if response.status_code == 200:
            complete_response = ""
            for line in response.iter_lines():
                if line:
                    try:
                        chunk = json.loads(line)
                        if "message" in chunk and "content" in chunk["message"]:
                            complete_response += chunk["message"]["content"]
                    except (KeyError, json.JSONDecodeError):
                        continue
            return complete_response
        else:
            return f"Error: {response.status_code} - {response.text}"
    except requests.exceptions.RequestException as e:
        print("Error connecting to the server:", e)
        return None

In [9]:
def evaluate_prompts(input_file, output_file, models, prompts, max_reviews=100):
    # Read the input data
    df = pd.read_csv(input_file)

    if 'review_content' not in df.columns:
        print("Error: 'review_content' column not found in the input file.")
        return None  # Return None in case of errors

    # Limit the dataset to the first `max_reviews` reviews
    df = df.head(max_reviews)

    # Initialize a results list
    results = []

    # Iterate through each model and prompt combination
    for model in models:
        for prompt_template in prompts:
            print(f"Evaluating model: {model} with prompt: {prompt_template[:50]}...")

            for index, row in df.iterrows():
                review_content = row['review_content']
                prompt = prompt_template.format(review=review_content)

                # Get the response from the model
                response = get_response_from_server(prompt, model)

                # Append results
                results.append({
                    "model": model,
                    "prompt": prompt_template,
                    "review_index": index,
                    "review_content": review_content,
                    "response": response
                })

    return results

In [10]:
input_file = "cleaned_sampled_reviews.csv"
output_file = "prompt_evaluation_results.csv"

In [11]:
models = ["gemma2:27b", "codellama:latest", "llama3.1:latest"]

In [12]:
prompts = [
    "Turn the following review into a concise user story: {review}",
    "Based on the review below, create a user story in the format: As a [User Role], I want [Goal] so that [Benefit]: {review}",
    "Generate a user story that summarizes the key features or improvements requested in this review: {review}"
]

In [14]:
results = evaluate_prompts(input_file, output_file, models, prompts, max_reviews=100)

if results:  # Only proceed if results are returned
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)
    print(f"Evaluation results saved to {output_file}")
else:
    print("No results to save.")

Evaluating model: gemma2:27b with prompt: Turn the following review into a concise user stor...
Evaluating model: gemma2:27b with prompt: Based on the review below, create a user story in ...
Evaluating model: gemma2:27b with prompt: Generate a user story that summarizes the key feat...
Evaluating model: codellama:latest with prompt: Turn the following review into a concise user stor...
Evaluating model: codellama:latest with prompt: Based on the review below, create a user story in ...
Evaluating model: codellama:latest with prompt: Generate a user story that summarizes the key feat...
Evaluating model: llama3.1:latest with prompt: Turn the following review into a concise user stor...
Evaluating model: llama3.1:latest with prompt: Based on the review below, create a user story in ...
Evaluating model: llama3.1:latest with prompt: Generate a user story that summarizes the key feat...
Evaluation results saved to prompt_evaluation_results.csv


In [ ]:
#print the column names 
print(results_df.columns)

In [21]:
import pandas as pd

# Load the CSV file
file_path = r"D:\Research\Thesis\prompt_evaluation_results.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Define the desired prompt order
prompt_order = [
    "Turn the following review into a concise user story: {review}",
    "Based on the review below, create a user story in the format: As a [User Role], I want [Goal] so that [Benefit]: {review}",
    "Generate a user story that summarizes the key features or improvements requested in this review: {review}"
]

# Function to map prompts to their order
def get_prompt_order(prompt):
    return prompt_order.index(prompt) if prompt in prompt_order else -1

# Create a new column 'prompt_order' to hold the order index for each prompt
df['prompt_order'] = df['prompt'].apply(get_prompt_order)

# Sort the DataFrame by 'model' first and then by 'prompt_order' to maintain the correct order within each model
df_sorted = df.sort_values(by=['model', 'prompt_order'])

# Drop the 'prompt_order' column as it's no longer needed
df_sorted = df_sorted.drop(columns=['prompt_order'])

# Save the sorted dataframe to a new CSV file
df_sorted.to_csv(r"D:\Research\Thesis\sorted_prompt_evaluation_results.csv", index=False)

# Display the first few rows of the sorted dataframe
print(df_sorted.head())


                model                                             prompt  \
300  codellama:latest  Turn the following review into a concise user ...   
301  codellama:latest  Turn the following review into a concise user ...   
302  codellama:latest  Turn the following review into a concise user ...   
303  codellama:latest  Turn the following review into a concise user ...   
304  codellama:latest  Turn the following review into a concise user ...   

     review_index                                     review_content  \
300             0  In addition to being notorious for it39s priva...   
301             1  Ive been trying to get a hold of Facebook beca...   
302             2  Keep getting banned on Facebook for random thi...   
303             3  I have been locked out of my account for 2 wee...   
304             4  For all the good Meta does in keeping us conne...   

                                              response  
300  As a user of Facebook, I want to be able to sh..